In [1]:
#Start Spark Session

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ClimateDataset").getOrCreate()
sc = spark.sparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1646524137094_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Read csv from S3 bucket

filepath="s3://trabajo1-st1800-st18101/dataset/raw/GlobalLandTemperaturesByMajorCity.csv"
df = spark.read.csv(filepath,inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Print the dataframe columns
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country', 'Latitude', 'Longitude']

In [4]:
#Count the number of records in the dataframe
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

239177

In [5]:
#Clean the dataframe, removing null values in the df_clean and adding another dataframe with those null values in df_nulls
df_clean = df.filter(df.AverageTemperature.isNotNull())
df_nulls = df.filter(df.AverageTemperature.isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#Count the records of the clean dataframe
df_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

228175

In [7]:
#Count the records of the dataframe with only null values
df_nulls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11002

In [8]:
#Show the records and the column names for the dataframe
df_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1849-01-01|            26.704|                        1.435|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-02-01|            27.434|                        1.362|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-03-01|            28.101|                        1.612|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-04-01|             26.14|           1.3869999999999998|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-05-01|            25.427|                          1.2|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-06-01|            24.844|                        1.402|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-07-01|24.058000000000003|                        1.254|Abidjan|Côte

In [9]:
from pyspark.sql.functions import avg, countDistinct, max
from pyspark.sql.functions import round as roundspark

df_clean = df_clean.withColumn("AverageTemperature",roundspark(df_clean['AverageTemperature'],2))
df_clean = df_clean.withColumn("AverageTemperatureUncertainty",roundspark(df_clean['AverageTemperatureUncertainty'],2))
    

df_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1849-01-01|              26.7|                         1.44|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-02-01|             27.43|                         1.36|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-03-01|              28.1|                         1.61|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-04-01|             26.14|                         1.39|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-05-01|             25.43|                          1.2|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-06-01|             24.84|                          1.4|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-07-01|             24.06|                         1.25|Abidjan|Côte

In [19]:

average_temperature_all = ["Average Temperatures",str(round(df_clean.select(avg("AverageTemperature")).first()[0],2))]
count_cities = ["Number of Cities",str(df_clean.select(countDistinct("City")).first()[0])]
count_country = ["Number of Countries",str(df_clean.select(countDistinct("Country")).first()[0])]
highest_temperature = ["Highest Temperature",str(df_clean.select(max(df_clean['AverageTemperature'])).first()[0])]
average_temperature_cali = ["Average Temp Cali",str(round(df_clean.filter(df["City"] == "Cali").agg(avg("AverageTemperature")).first()[0],2))]

columns = ['Name','Value']
vals = [(average_temperature_all),(count_cities),(count_country),
       (highest_temperature), (average_temperature_cali)]

df_stats = spark.createDataFrame(vals, columns)

df_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                Name|Value|
+--------------------+-----+
|Average Temperatures|18.13|
|    Number of Cities|  100|
| Number of Countries|   49|
| Highest Temperature|38.28|
|   Average Temp Cali| 21.8|
+--------------------+-----+

In [20]:
#Write the clean dataframe into S3 as a csv
df_clean.write.csv("s3://trabajo1-st1800-st18101/dataset/trusted/",mode="overwrite", header=True)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Write the refined dataframe with some statistics
df_stats.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("s3://trabajo1-st1800-st18101/dataset/refined/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…